Imports

In [57]:
import numpy as np

todo: initialise constants from the dataset (drop name column)
todo: redefine feed forward generally
todo: do todo 

Intialise network architecture

In [58]:
L = 3  # number of layers(excluding input layer)
n = 2  # number of features
m = 1  # number of output nodes

hidden_layers = 2  # number of layers(excluding input layer)
features = 2  # number of features
outputs = 1  # number of output nodes

In [59]:
def init_network(input_size=features, hidden_size=features+1, output_size=outputs, num_hidden_layers=hidden_layers):
    weights = []
    biases = []
    
    # Input to first hidden layer
    weights.append(np.random.randn(hidden_size, input_size) * 0.01)
    biases.append(np.random.randn(hidden_size, 1) * 0.01)
    
    # Hidden layers
    for i in range(num_hidden_layers - 1):
        weights.append(np.random.randn(hidden_size, hidden_size) * 0.01)
        biases.append(np.random.randn(hidden_size, 1) * 0.01)
    
    # Last hidden to output layer
    weights.append(np.random.randn(output_size, hidden_size) * 0.01)
    biases.append(np.random.randn(output_size, 1) * 0.01)
    
    return weights, biases

In [60]:
weights, biases = init_network()
print("Weights of the network:", weights)
print("Biases of the network:", biases)

Weights of the network: [array([[-0.0065811 , -0.00094568],
       [ 0.01696198,  0.00922171],
       [ 0.00129864, -0.0092376 ]]), array([[ 1.63511434e-02,  1.12045895e-03,  7.05345569e-03],
       [ 2.45510363e-05, -7.49741630e-03,  3.09564123e-03],
       [ 6.39512349e-03,  3.36422767e-03, -1.52499969e-02]]), array([[-0.01222086,  0.0009136 ,  0.00015016]])]
Biases of the network: [array([[0.01409215],
       [0.0036405 ],
       [0.00611037]]), array([[ 0.00272478],
       [-0.01465117],
       [-0.00879309]]), array([[0.01159015]])]


Training data

In [61]:
def prepare_data():
  X = np.array([
      [150, 70],
      [254, 73],
      [312, 68],
      [120, 60],
      [154, 61],
      [212, 65],
      [216, 67],
      [145, 67],
      [184, 64],
      [130, 69]
  ])
  y = np.array([0,1,1,0,0,1,1,0,1,0])
  m = y.shape[0]  # number of samples
  # Transpose the input matrix to match the expected shape
  A0 = X.T
  # Reshape y to a matrix
  Y = y.reshape(outputs, m)

  return A0, Y

In [62]:
x_train, y_train = prepare_data()

Activation function sigmoid 
$$g(z)=\frac{1}{(1+e^{-z})}$$

In [63]:
# Sigmoid activation function
def sigmoid(arr):
    return 1 / (1+np.exp(-1*arr))

Feed forward

In [64]:
def feed_forward(input_layer=x_train, num_hidden_layers=hidden_layers, weights=weights, biases=biases):
  cache = [input_layer]
  
  for i in range(0,num_hidden_layers+1):
    prev_layer = cache[i]
    W = weights[i]
    b = biases[i]
    Z = W @ prev_layer + b
    A = sigmoid(Z)
    cache.append(A)

  return cache


In [65]:
cache = feed_forward(x_train)  # Get the output layer predictions
predictions = cache[-1]  # Last layer output
print("Predictions of the network:", cache[-1])
print("\nPredictions:", cache)

Predictions of the network: [[0.50149336 0.50149455 0.50149491 0.5014928  0.50149328 0.50149403
  0.5014941  0.50149325 0.5014937  0.50149307]]

Predictions: [array([[150, 254, 312, 120, 154, 212, 216, 145, 184, 130],
       [ 70,  73,  68,  60,  61,  65,  67,  67,  64,  69]]), array([[0.26129076, 0.1510304 , 0.10875538, 0.30314191, 0.25786712,
        0.1911476 , 0.18682313, 0.26824833, 0.22142972, 0.287675  ],
       [0.96058735, 0.99320783, 0.99732961, 0.93036965, 0.96000986,
        0.9852113 , 0.98641732, 0.95610147, 0.97621982, 0.94505948],
       [0.39037734, 0.41620278, 0.44599345, 0.40316148, 0.41159069,
        0.42091388, 0.41768022, 0.39543883, 0.41431704, 0.38640206]]), array([[0.50270672, 0.50231069, 0.50219156, 0.50289187, 0.50272997,
        0.50248074, 0.5024577 , 0.50274283, 0.50259037, 0.50280321],
       [0.49484063, 0.4947988 , 0.49481387, 0.49490741, 0.49485811,
        0.49481768, 0.49481289, 0.494853  , 0.49482961, 0.49486682],
       [0.49753909, 0.49729179, 0.

Use loss as cost function: $L\left(\hat{y}_i y_i\right)=-\left(y_i \ln \hat{y}_i+\left(1-y_i\right) \ln \left(1-\hat{y}_i\right)\right)$


Hence cost given by: $C=\frac{1}{m} \sum_{i=1}^m L\left(\hat{y}_i, y_i\right)$

In [66]:
def cost(y_hat=predictions, y=y_train):
  """
  y_hat should be a n^L x m matrix
  y should be a n^L x m matrix
  """

  # Clip predictions to avoid log(0)
  y_hat = np.clip(y_hat, 1e-15, 1 - 1e-15)

  # 1. losses is a n^L x m
  losses = - ( (y * np.log(y_hat)) + (1 - y)*np.log(1 - y_hat) )

  return np.mean(losses)

In [67]:
print(cost(predictions, y_train))

0.6931505366938915


Backpropagation with formulas derived using calculus

In [72]:
def backprop_layer(dC_dA_current, A_current, A_prev, W_current, Y, layer_num):
    """
    Backpropagation for a single layer.

    Args:
        dC_dA_current:  Propagator from the next layer (dC/dA[l+1]). For the output layer, this is None.
        A_current:      Activations of the current layer (A[l])
        A_prev:         Activations of the previous layer (A[l-1])
        W_current:      Weights of the current layer (W[l])
        Y:              True labels (only needed for the output layer)

    Returns:
        dC_dW: Gradient of cost with respect to weights
        dC_db: Gradient of cost with respect to biases
        dC_dA_prev: Propagator to the previous layer (dC/dA[l-1])
    """

    # Calculate dC/dZ
    if dC_dA_current is None:  # Output layer
        dC_dZ = (1/m) * (A_current - Y)
    else:  # Hidden layer
        dA_dZ = A_current * (1 - A_current)
        dC_dZ = dC_dA_current * dA_dZ


    # calculate dC/dW
    dZ_dW = A_prev
    dC_dW = dC_dZ @ dZ_dW.T

    # step 3. calculate dC/db
    dC_db = np.sum(dC_dZ, axis=1, keepdims=True)

    # calculate propagator to the prev layer
    if layer_num != 1:
        dZ_dA_prev = W_current
        dC_dA_prev = W_current.T @ dC_dZ

    else:
        dC_dA_prev = None

    return dC_dW, dC_db, dC_dA_prev

Train

In [73]:
def train(epochs=200, alpha=0.1, hidden_layers=hidden_layers, features=features, outputs=outputs):
  global weights, biases, x_train, y_train
  
  costs = []

  for e in range(epochs):
    # Feed forward
    cache = feed_forward(x_train)
    predictions = cache[-1]
    
    # Cost
    costs.append(cost(predictions, y_train))

    # Backpropagation 
    updates = []
    for layer_num in range(hidden_layers, 0, -1):
        if layer_num == hidden_layers:
            # For the output layer, we need to pass the true labels Y
            propagations = backprop_layer(
                dC_dA_current=None, 
                A_current=cache[layer_num], 
                A_prev=cache[layer_num-1], 
                W_current=weights[layer_num], 
                Y=y_train,
                layer_num=layer_num
            )
        else:
            # For hidden layers, we pass the propagator from the next layer
            propagations = backprop_layer(
                dC_dA_current=updates[0], 
                A_current=cache[layer_num], 
                A_prev=cache[layer_num-1], 
                W_current=weights[layer_num],
                layer_num=layer_num
            )

        updates.insert(0, propagations)

    # Update weights and biases
    for layer_num in range(hidden_layers, -1, -1):
        dC_dW, dC_db, dC_dA_prev = updates[layer_num]

        # Update weights and biases
        weights[layer_num] -= alpha * dC_dW
        biases[layer_num] -= alpha * dC_db


    if e % 20 == 0:
      print(f"epoch {e}: cost = {costs[e]:4f}")
  
  return costs

In [70]:
for i in range(10,1,-1):
  print(i)

10
9
8
7
6
5
4
3
2


In [74]:
costs = train()

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 1)